In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py

import tensorflow as tf
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices(device_type="GPU")[0], True)
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from tqdm import tqdm

from msfm.utils import tfrecords, files, parameters, cross_statistics
from msfm.fiducial_pipeline import FiducialPipeline
from msfm.grid_pipeline import GridPipeline
from msfm.utils.input_output import read_yaml

In [3]:
with_lensing = True
with_clustering = True

# fiducial

In [6]:
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_0000.tfrecord"
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_0001.tfrecord"
# tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_00??.tfrecord"
tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/fiducial/DESy3_fiducial_dmb_0000.tfrecord"

conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias_debug.yaml"
# conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias.yaml"
conf = files.load_config(conf)

# params = ["Om", "s8", "Ob", "H0", "ns", "w0", "Aia", "n_Aia", "bg", "n_bg"]
# params = ["Om", "s8"]
params = ["Om", "Aia"]
# params = ["s8", "Aia"]
# params = ["Om", "bg"]
# params = ["Om"]
# params = ["Aia"]
# params = ["bg"]

In [7]:
fidu_pipe = FiducialPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
)

data_vec_pix = fidu_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = fidu_pipe.n_z_metacal + fidu_pipe.n_z_maglim
n_z_metacal = fidu_pipe.n_z_metacal
n_noise = conf["analysis"]["fiducial"]["n_noise_per_example"]
fidu_batch_size = 16

fidu_dset = fidu_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=fidu_batch_size,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

24-08-16 00:14:01     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:14:01     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:14:01 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-16 00:14:01 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-16 00:14:01 fiducial_pip INF   Including noise_indices = [0, 1] 
24-08-16 00:14:01 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-16 00:14:01 fiducial_pip INF   Interleaving with n_readers = 1 
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:8", shape=(), dtype=int64)
24-08-16 00:14:02 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-16 00:14:02 fiducial_pip INF   Batching into 16 elements locally with drop_

In [8]:
%%time
n_batches = 100
for x in tqdm(fidu_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [02:13<00:00,  1.33s/it]

CPU times: user 22min 11s, sys: 8min 13s, total: 30min 24s
Wall time: 2min 13s


In [9]:
fidu_pipe = FiducialPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=False,
)

data_vec_pix = fidu_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = fidu_pipe.n_z_metacal + fidu_pipe.n_z_maglim
n_z_metacal = fidu_pipe.n_z_metacal
n_noise = conf["analysis"]["fiducial"]["n_noise_per_example"]
fidu_batch_size = 16

fidu_dset = fidu_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=fidu_batch_size,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

24-08-16 00:16:17     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:16:18     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:16:18 fiducial_pip INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-16 00:16:18 fiducial_pip INF   drop_remainder is not set, using drop_remainder = True 
24-08-16 00:16:18 fiducial_pip INF   Including noise_indices = [0, 1] 
24-08-16 00:16:18 fiducial_pip INF   Shuffling file names with shuffle_buffer = 16 
24-08-16 00:16:18 fiducial_pip INF   Interleaving with n_readers = 1 
Tensor("ParseSingleExample/ParseExample/ParseExampleV2:5", shape=(), dtype=int64)
24-08-16 00:16:18 fiducial_pip INF   Shuffling examples with shuffle_buffer = 64 
24-08-16 00:16:18 fiducial_pip INF   Batching into 16 elements locally with drop_

In [10]:
%%time
n_batches = 100
for x in tqdm(fidu_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [00:44<00:00,  2.23it/s]

CPU times: user 1min 5s, sys: 1min 18s, total: 2min 23s
Wall time: 44.8 s


# grid

In [4]:
tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v10/debug/linear_bias/tfrecords/grid/DESy3_grid_dmb_0000.tfrecord"

conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v10/linear_bias_debug.yaml"
conf = files.load_config(conf)

params = ["Om", "s8", "Ob", "H0", "ns", "w0", "Aia", "n_Aia", "bg", "n_bg"]

In [24]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=True,
)

data_vec_pix = grid_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = grid_pipe.n_z_metacal + grid_pipe.n_z_maglim
n_z_metacal = grid_pipe.n_z_metacal
n_noise = conf["analysis"]["grid"]["n_noise_per_example"]

grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=128,
    n_readers=1,
    n_prefetch=0,
    is_eval=False,
)

24-08-15 23:51:20     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-15 23:51:20     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-15 23:51:20 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-15 23:51:20 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-15 23:51:20 grid_pipelin INF   Including noise_indices = [0, 1] 
24-08-15 23:51:20 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-15 23:51:20 grid_pipelin INF   Interleaving with n_readers = 1 
24-08-15 23:51:20 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-15 23:51:20 grid_pipelin INF   Batching into 128 elements locally 
24-08-15 23:51:20 grid_pipelin WAR   Tracing _augmentations 
24-08-15 23:51:20 grid_pipe

In [ ]:
%%time
n_batches = 100
for x in tqdm(grid_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [03:35<00:00,  2.14s/it]

In [10]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=with_lensing,
    with_clustering=with_clustering,
    with_padding=True,
    apply_norm=False,
    return_maps=False,
)

data_vec_pix = grid_pipe.data_vec_pix
n_side = conf["analysis"]["n_side"]
n_pix = hp.nside2npix(n_side)
n_z = grid_pipe.n_z_metacal + grid_pipe.n_z_maglim
n_z_metacal = grid_pipe.n_z_metacal
n_noise = conf["analysis"]["grid"]["n_noise_per_example"]

grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=128,
    n_readers=1,
    n_prefetch=0,
    is_eval=False
)

24-08-16 00:02:00     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:02:01     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_fiducial_512.h5 
24-08-16 00:02:01 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-08-16 00:02:01 grid_pipelin INF   drop_remainder is not set, using drop_remainder = True 
24-08-16 00:02:01 grid_pipelin INF   Including noise_indices = [0, 1] 
24-08-16 00:02:01 grid_pipelin INF   Shuffling file names with shuffle_buffer = 128 
24-08-16 00:02:01 grid_pipelin INF   Interleaving with n_readers = 1 
cosmo
i_sobol
i_example
24-08-16 00:02:01 grid_pipelin INF   Shuffling examples with shuffle_buffer = 128 
24-08-16 00:02:01 grid_pipelin INF   Batching into 128 elements locally 
24-08-16 00:02:01 grid_pipelin WAR   Tracing _augmentations 
24-

In [11]:
%%time
n_batches = 100
for x in tqdm(grid_dset.take(n_batches), total=n_batches):
    pass

100%|██████████| 100/100 [00:58<00:00,  1.70it/s]

CPU times: user 1min 2s, sys: 38.6 s, total: 1min 40s
Wall time: 58.7 s
